In [ ]:
!pip install keras_preprocessing # install keras_preprocessing package
import tensorflow as tf
import keras_preprocessing #import keras_preprocessing
from tensorflow.keras.preprocessing import image
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Instead of importing TensorBoard from keras.callbacks, we will use it from tf.keras.callbacks
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential # Changed from keras.models to tensorflow.keras.models
# Import layers from tensorflow.keras.layers
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
# BatchNormalization is now imported directly from tensorflow.keras.layers
#from keras.layers.normalization import BatchNormalization # This line is removed
from tensorflow.keras.optimizers import Adam # Changed from keras.optimizers to tensorflow.keras.optimizers
import keras
from tensorflow.keras.callbacks import ModelCheckpoint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

train_dir = "/content/drive/My Drive/potato dis/train"
test_dir = "/content/drive/My Drive/potato dis/test"
validation_dir = "/content/drive/My Drive/potato dis/val"

In [ ]:
import os

dataset_path = train_dir
class_names = os.listdir(dataset_path)
print("Classes:", class_names)


Classes: ['Potato___Late_blight', 'Potato___healthy', 'Potato___Early_blight']


In [ ]:
ROT_RANGE = 10

# this is the augmentation configuration we will use for training
train_gen = ImageDataGenerator(
rescale = 1./255,
rotation_range = ROT_RANGE,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

valid_gen = ImageDataGenerator(rescale = 1./255)


In [ ]:

TARGET_SIZE = (224,224)
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
SEED = 42

#Data Iterator
train_data = train_gen.flow_from_directory(
train_dir,
target_size = TARGET_SIZE,
class_mode = 'categorical',
color_mode = "rgb",
batch_size = TRAIN_BATCH_SIZE,
shuffle = True,
seed = SEED
)

valid_data = valid_gen.flow_from_directory(
validation_dir,
target_size = TARGET_SIZE,
class_mode = 'categorical',
color_mode = "rgb",
batch_size = VALID_BATCH_SIZE
)

Found 1506 images belonging to 3 classes.
Found 215 images belonging to 3 classes.


In [ ]:
from tensorflow import keras
base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=[224, 224, 3])


x = keras.layers.Flatten() (base_model.output)
x = keras.layers.Dense(256, activation="relu")(x)
x = keras.layers.Dropout(0.25)(x)
output = keras.layers.Dense(3, activation='softmax')(x)
model = keras.models.Model(inputs=base_model.input, outputs=output)

# The newly added layers are initialized with random values.
# Make sure based model remain unchanged until newly added layers weights get reasonable values.
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
LEARNING_RATE = 0.0001
#LEARNING_RATE = 0.001

# Change 'lr' to 'learning_rate'
opt = Adam(learning_rate = LEARNING_RATE)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
history = model.fit(x = train_data, validation_data=valid_data,epochs=10)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


48/48 ━━━━━━━━━━━━━━━━━━━━ 1316s 24s/step - accuracy: 0.6483 - loss: 0.8216 - val_accuracy: 0.8837 - val_loss: 0.3308
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 53s 471ms/step - accuracy: 0.8740 - loss: 0.3730 - val_accuracy: 0.9256 - val_loss: 0.2539
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 26s 483ms/step - accuracy: 0.9004 - loss: 0.2913 - val_accuracy: 0.9535 - val_loss: 0.1890
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 41s 475ms/step - accuracy: 0.9062 - loss: 0.2407 - val_accuracy: 0.9535 - val_loss: 0.1746
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 41s 459ms/step - accuracy: 0.9274 - loss: 0.2194 - val_accuracy: 0.9721 - val_loss: 0.1540
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 26s 469ms/step - accuracy: 0.9402 - loss: 0.1777 - val_accuracy: 0.9442 - val_loss: 0.1540
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 26s 478ms/step - accuracy: 0.9288 - loss: 0.1891 - val_accuracy: 0.9581 - val_loss: 0.1472
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 27s 489ms/step - accuracy: 0.9391 - loss: 0.1736 - val_accuracy: 0.962

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate (This ensures the base model weights do not change a lot)
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [ ]:
history = model.fit(x = train_data, validation_data=valid_data,epochs=5)

Epoch 1/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 35s 637ms/step - accuracy: 0.9919 - loss: 0.0336 - val_accuracy: 0.9860 - val_loss: 0.0374
Epoch 2/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 40s 616ms/step - accuracy: 0.9968 - loss: 0.0166 - val_accuracy: 0.9814 - val_loss: 0.0519
Epoch 3/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 41s 619ms/step - accuracy: 0.9922 - loss: 0.0202 - val_accuracy: 0.9767 - val_loss: 0.0960
Epoch 4/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 42s 671ms/step - accuracy: 0.9902 - loss: 0.0253 - val_accuracy: 0.9907 - val_loss: 0.0180
Epoch 5/5
48/48 ━━━━━━━━━━━━━━━━━━━━ 34s 634ms/step - accuracy: 0.9899 - loss: 0.0344 - val_accuracy: 0.9907 - val_loss: 0.0199


In [ ]:
from tensorflow.keras.models import save_model
model.save("/content/drive/My Drive/potato dis/vgg16(2).h5")
  # Save the model in HDF5 format


In [ ]:
from google.colab import files
files.download("/content/drive/My Drive/potato dis/vgg16(1).h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import tensorflow as tf
print(tf.__version__)  # Example: 2.14.0


2.17.1
